<a href="https://colab.research.google.com/github/mary-ts/Neural-Computing/blob/tito/itworksispose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.layers as layers
from keras import Sequential
import keras.models
from keras.layers import *
from keras.models import * 
from keras import optimizers, applications
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

Dataset Paths

In [4]:
train_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/train/train"
test_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/test/test"

Creating training and test data

In [5]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split = 0.2
)
test_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

Importing data into generators

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (299,299),
    batch_size = 32,
    class_mode = 'categorical', 
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (299,299),
    batch_size = 32,
    shuffle=True,
    class_mode = 'categorical')

Found 16854 images belonging to 33 classes.
Found 0 images belonging to 0 classes.


Xception Model

In [21]:
base_model = applications.Xception(weights='imagenet', 
                              include_top=False, 
                              input_shape=(299, 299, 3))

for layer in base_model.layers[:-5]:
    layer.trainable = False

model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation="relu"))
model.add(keras.layers.Dense(33, activation="softmax"))

input_shape = (None, 299, 299, 3)
model.build(input_shape)
model.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-5),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 flatten_10 (Flatten)        (None, 204800)            0         
                                                                 
 dense_13 (Dense)            (None, 1024)              209716224 
                                                                 
 dense_14 (Dense)            (None, 33)                33825     
                                                                 
Total params: 230,611,529
Trainable params: 212,916,769
Non-trainable params: 17,694,760
_________________________________________________________________


Xception


In [22]:
checkpoint = ModelCheckpoint("xception", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='loss', min_delta=0.0001, patience=7, verbose=2, mode='auto')
hist = model.fit_generator(generator=train_generator,                   
                                    steps_per_epoch=150,
                                    validation_data=test_generator,                    
                                    validation_steps=10,
                                    epochs=5,
                                    callbacks = [checkpoint, early],
                                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
Y_pred = model.predict_generator(test_generator, 10 // 150+1)
y_pred = np.argmax(Y_pred, axiz=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Apple Braeburn',
'Apple Granny Smith',
'Apricot',
'Avocado',
'Banana',
'Blueberry',
'Cactus fruit',
'Cantaloupe',
'Cherry',
'Clementine',
'Corn',
'Cucumber RIpe',
'Grape blue',
'Kiwi',
'Lemon',
'Limes',
'Mango',
'Onion White',
'Orange',
'Papaya',
'Passion fruit',
'Peach',
'Pear',
'Pepper Green',
'Pepper Red',
'Pineapple',
'Plum',
'Pomegranate',
'Potato Red',
'Raspberry',
'Strawberry',
'Tomato',
'Watermelon']

Plot Xception

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["loss"])
plt.plot(hist.history["accuracy"])
#plt.plot(hist.history["val_loss"])
#plt.plot(hist.history["val_accuracy"])
plt.legend(["loss", "val_accuracy"])
plt.show()